In [1]:
import requests

def fetch_cves(limit=10000):
    base_url = "https://access.redhat.com/labs/securitydataapi/cve.json"
    cves = []
    page = 1
    per_page = 100  # Number of CVEs per page (max allowed by the API)

    while len(cves) < limit:
        url = f"{base_url}?page={page}&per_page={per_page}"
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break
        
        data = response.json()
        if not data:
            break  # No more data available
        
        cves.extend(data)
        page += 1

        print(f"Fetched {len(cves)} CVEs so far...")

        if len(data) < per_page:
            break  # No more pages available

    return cves[:limit]

# Fetch the first 20,000 CVEs
cves = fetch_cves(limit=20000)

# Print the total number of CVEs fetched
print(f"Total CVEs fetched: {len(cves)}")

Fetched 100 CVEs so far...
Fetched 200 CVEs so far...
Fetched 300 CVEs so far...
Fetched 400 CVEs so far...
Fetched 500 CVEs so far...
Fetched 600 CVEs so far...
Fetched 700 CVEs so far...
Fetched 800 CVEs so far...
Fetched 900 CVEs so far...
Fetched 1000 CVEs so far...
Fetched 1100 CVEs so far...
Fetched 1200 CVEs so far...
Fetched 1300 CVEs so far...
Fetched 1400 CVEs so far...
Fetched 1500 CVEs so far...
Fetched 1600 CVEs so far...
Fetched 1700 CVEs so far...
Fetched 1800 CVEs so far...
Fetched 1900 CVEs so far...
Fetched 2000 CVEs so far...
Fetched 2100 CVEs so far...
Fetched 2200 CVEs so far...
Fetched 2300 CVEs so far...
Fetched 2400 CVEs so far...
Fetched 2500 CVEs so far...
Fetched 2600 CVEs so far...
Fetched 2700 CVEs so far...
Fetched 2800 CVEs so far...
Fetched 2900 CVEs so far...
Fetched 3000 CVEs so far...
Fetched 3100 CVEs so far...
Fetched 3200 CVEs so far...
Fetched 3300 CVEs so far...
Fetched 3400 CVEs so far...
Fetched 3500 CVEs so far...
Fetched 3600 CVEs so far...
F

In [2]:
len(cves)

20000

In [3]:
def extract_year_from_cve(cve_id):
    """
    Extracts the year from a CVE identifier.

    :param cve_id: A string representing the CVE ID (e.g., "CVE-2022-49043").
    :return: The year as an integer (e.g., 2022).
    """
    # Split the CVE ID by '-' and get the second part (the year)
    try:
        year = int(cve_id.split('-')[1])
        return year
    except (IndexError, ValueError) as e:
        print(f"Error extracting year from CVE ID: {e}")
        return None

In [4]:
final_packages = []
final_cves = []
fixed_states = []

In [5]:
def extract_from_openshift4_onwards(input_string):
    """
    Extracts all the data from 'openshift4/' onwards in the input string.

    :param input_string: The input string (e.g., "red_hat_openshift_container_platform_4:openshift4/oc-mirror-plugin-rhel8").
    :return: The substring starting from 'openshift4/' onwards. Returns None if 'openshift4/' is not found.
    """
    # Find the index of 'openshift4/'
    index = input_string.find('openshift4/')
    
    # If 'openshift4/' is found, extract the substring from that point onwards
    if index != -1:
        return input_string[index:]
    else:
        return None

In [6]:
for cve in cves:
    cve_id = cve['CVE'].lower()
    year = extract_year_from_cve(cve_id)
    resource_url = f"https://security.access.redhat.com/data/csaf/v2/vex/{year}/{cve_id}.json"
    response = requests.get(resource_url)
    if response.status_code != 200:
        continue
    data = response.json()
    for product in data.get("vulnerabilities", []):
        for remediation in product.get("remediations", []):
            if remediation["category"] == 'none_available' and remediation["details"] == 'Affected':
                packages = remediation["product_ids"]
                for package in packages:
                    if "openshift4/" in package:
                        final_packages.append(extract_from_openshift4_onwards(package))
                        final_cves.append(cve['CVE'])
                        fixed_states.append("known_affected")


In [7]:
len(cves)

20000

In [8]:
len(final_cves), len(final_packages)

(673, 673)

In [9]:
from pandas import DataFrame
df = DataFrame({'CVE': final_cves, 'package': final_packages, 'Fix State': fixed_states})
df.to_excel('known_affected.xlsx', sheet_name='sheet1', index=False)